# Distribution Approach
Interferometry tracks the ToF, giving us an intensity distribution over time. Assuming the refractive indices are similar across the entire tissue structure/body, photon total path ($\sum_{i=1}^{T}L_i$; with T being the number of tissue layers) can be a proxy for time. (If the refractive indices are different, we need to calculate the speed of light in each tissue layer/medium separately and then calculate the actual ToF for each photon. Although that approach is tractable in our setup, it's simpler to assume the speed of light remains constant for now.) So, the simulation plots we are interested in are Intensity($<G, A>$) vs. Photon total path($\sum_{i=1}^{T}L_i$)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from inverse_modelling_tfo.tools.name_decoder import decode_extended_filename
from ipywidgets import interact
import ipywidgets as widgets

# Pre-calculated intensity using the regular formula (inner product)
INTENSITY_DATA_PATH = Path('../data/s_based_intensity_low_conc2.pkl')

intensity_data = pd.read_pickle(INTENSITY_DATA_PATH)
RAW_SIM_DATA_PATH = '/home/rraiyan/simulations/tfo_sim/data/raw_dan_iccps_equispace_detector/fa_1_wv_1_sa_0.1_ns_1_ms_2_ut_5.pkl'
raw_sim_data = pd.read_pickle(RAW_SIM_DATA_PATH)
maternal_wall_thickness, uterus_thickness, wave_int = decode_extended_filename(RAW_SIM_DATA_PATH)
intensity_data = intensity_data[(intensity_data['Maternal Wall Thickness'] == maternal_wall_thickness) & (intensity_data['Wave Int'] == wave_int)]

In [8]:
intensity_data.groupby('Fetal Hb Concentration').groups.keys()

dict_keys([0.11, 0.125, 0.14, 0.15500000000000003, 0.17])

In [2]:
from inverse_modelling_tfo.tools.s_based_intensity_datagen import MU_MAP_BASE1, MU_MAP_BASE2, get_mu_a
# Create SDD column!
raw_sim_data['SDD'] = raw_sim_data['X'] - 100
raw_sim_data['Total Path'] = raw_sim_data['L1 ppath'] + raw_sim_data['L2 ppath'] + raw_sim_data['L3 ppath'] + raw_sim_data['L4 ppath']

In [15]:
all_sdd = raw_sim_data['SDD'].unique()

def plot_L_distribution(sdd_index):
    SDD = all_sdd[sdd_index]
    plt.figure()
    plt.hist(raw_sim_data[raw_sim_data['SDD'] == SDD]["Total Path"], bins=100)
    plt.xlabel("Total Path (mm)")
    plt.ylabel("Count")

sddSlider = widgets.IntSlider(15, min=0, max=19, step=1)
interact(plot_L_distribution, sdd_index=sddSlider)

interactive(children=(IntSlider(value=15, description='sdd_index', max=19), Output()), _dom_classes=('widget-i…

<function __main__.plot_L_distribution(sdd_index)>

In [11]:
# Mu Map
MATERNAL_Hb = 12.
MATERNAL_SAT = 0.9
FETAL_SAT = 0.225
FETAL_Hb = 0.11

DIGITIZATION_BIN_COUNT = 200

modified_mu_map = MU_MAP_BASE1.copy() if wave_int == 1 else MU_MAP_BASE2
modified_mu_map[1] = get_mu_a(MATERNAL_SAT, MATERNAL_Hb, wave_int)
epsilon = get_mu_a(FETAL_SAT, FETAL_Hb, wave_int) / FETAL_Hb
all_sdd = raw_sim_data['SDD'].unique()

def plot_distribution(c: float, sdd_index: int):
    SDD = all_sdd[sdd_index]
    filtered_photon_data = raw_sim_data[raw_sim_data['SDD'] == SDD].copy()
    G = filtered_photon_data[['L1 ppath', 'L2 ppath', 'L3 ppath']].to_numpy()
    for i in range(1, 4):
        G[:, i - 1] = np.exp(-modified_mu_map[i] * G[:, i - 1])
    G = np.prod(G, axis=1)
    L = filtered_photon_data['L4 ppath'].to_numpy()
    transformed_L = np.exp(- epsilon * c * L)
    I = G * transformed_L
    filtered_photon_data["Intensity"] = I
    # bins = np.linspace(0, stop=np.max(I), endpoint=True, num=DIGITIZATION_BIN_COUNT)
    intensity_v_L = filtered_photon_data.groupby("Total Path")["Intensity"].sum()
    plt.figure(figsize=(16,8))
    plt.plot(intensity_v_L)
    plt.title("Intensity v. Total Path Length")    
    plt.xlabel("Total Path(mm)")
    plt.yscale('log')
    plt.ylabel("Intensity")


cSlider = widgets.FloatSlider(4., min=0, max=15, step=0.1)
sddSlider = widgets.IntSlider(15, min=0, max=19, step=1)
interact(plot_distribution, c=cSlider, sdd_index=sddSlider)


interactive(children=(FloatSlider(value=4.0, description='c', max=15.0), IntSlider(value=15, description='sdd_…

<function __main__.plot_distribution(c: float, sdd_index: int)>

In [7]:
raw_sim_data['SDD'].unique()

array([10., 14., 19., 28., 23., 32., 46., 50., 37., 41., 55., 64., 59.,
       68., 91., 86., 73., 77., 82., 95.])

In [26]:
%matplotlib widget